In [1]:
#pip install obspy

In [1]:
import numpy as np
import obspy
from obspy import read
import pandas as pd
import time
import scipy

from __future__ import print_function
import keras
from keras.layers import add, Reshape, Dense,Input, TimeDistributed, Dropout, Activation, LSTM, Conv2D, Bidirectional, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping,ReduceLROnPlateau
from keras.regularizers import l1
from keras import backend as K
from keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from scipy import signal
import os
np.seterr(divide='ignore', invalid='ignore')
import h5py
from obspy.signal.trigger import trigger_onset
np.warnings.filterwarnings('ignore')

## Data Preprocess
### Stream generation (no parameter changes)

In [2]:
noise_filename = pd.read_csv('noise.csv', header = None)
noise_stream = []
for i in noise_filename[0]:
    temp = obspy.read("data/noise/" + i)
    noise_stream.append(temp)
    
signal_filename = pd.read_csv('signal.csv', header = None)
signal_stream = []
for i in signal_filename[0]:
    temp = obspy.read("data/signal/" + i)
    signal_stream.append(temp)

In [3]:
def slice_stream(st):
    start_time = st[0].stats.starttime
    end_time = st[0].stats.endtime
    gap = (end_time - start_time)/4
    st = st.slice(start_time + gap, end_time - gap)
    return st

In [4]:
noise_stream = list(map(slice_stream, noise_stream))
signal_stream = list(map(slice_stream, signal_stream))

In [5]:
noise_stream = [n.filter("bandpass", freqmin=1, freqmax=45) for n in noise_stream]
noise_stream = [n.resample(100) for n in noise_stream]
noise_stream = [n.detrend("demean") for n in noise_stream]
noise_stream = [n.normalize() for n in noise_stream]

In [6]:
signal_stream = [n.filter("bandpass", freqmin=1, freqmax=45) for n in signal_stream]
signal_stream = [n.resample(100) for n in signal_stream]
signal_stream = [n.detrend("demean") for n in signal_stream]
signal_stream = [n.normalize() for n in signal_stream]

In [57]:
length = signal_stream[0][0].stats.endtime - signal_stream[0][0].stats.starttime
signal_stream[0][0].stats.starttime + length/2 -(signal_stream[0][0].stats.starttime + 36*length/61)

-2.884344

### STFT 

In [8]:
f, t, Zxx = signal.stft(signal_stream[0], fs = 100, nperseg=25)

In [9]:
Zxx = np.abs(Zxx).T
Zxx.shape

(248, 13, 3)

In [11]:
noise_length = len(noise_stream)
signal_length = len(signal_stream)

f_signal = []
t_signal = []
Zxx_signal = np.zeros(shape = (signal_length, 248, 13, 3))

for i in range(signal_length):
    f, t, Zxx = signal.stft(signal_stream[i], fs = 100, nperseg=25)
    f_signal.append(f)
    t_signal.append(t)
    Zxx_signal[i] = np.abs(Zxx).T

f_noise = []
t_noise = []
Zxx_noise = np.zeros(shape = (noise_length, 248, 13, 3))

for i in range(noise_length):
    f, t, Zxx = signal.stft(noise_stream[i], fs = 100, nperseg=25)
    f_noise.append(f)
    t_noise.append(t)
    Zxx_noise[i] = np.abs(Zxx).T

### ground truth generation (Change dimensions)



In [71]:
## the time lap of the data is about 32 seconds and we would like to set the value appox. 0.5 sec before p-wave arrival
## and approx. 3 sec after p-wave arrival to be 1 and others be 0

noise_y = np.zeros(shape = (noise_length, 62, 1))

signal_y_truth = np.append(np.append(np.zeros(shape=(31,1)), np.ones(shape=(6,1)),axis=0), np.zeros(shape=(25,1)),axis=0)
signal_y = np.zeros(shape = (signal_length, 62, 1))
for i in range(signal_length):
    signal_y[i] = signal_y_truth

In [91]:
np.argwhere(signal_y_truth==1)[0][0]

31

### Combine noise and signal

In [73]:
y = np.append(noise_y, signal_y, axis = 0)
X_Zxx = np.append(Zxx_noise, Zxx_signal, axis = 0)

### Shuffle and split

In [74]:
print(y.shape)
print(X_Zxx.shape)

(19073, 62, 1)
(19073, 248, 13, 3)


In [75]:
np.random.seed(15)
X_1, X1_val, y_1, y1_val = train_test_split(X_Zxx, y, test_size=0.3, random_state=15,shuffle=True)

## CRED Model Training

In [78]:
def lr_schedule(epoch):
    """
    Learning rate is scheduled to be reduced after 40, 60, 80, 90 epochs.
    """
    lr = 1e-3
    if epoch > 60:
        lr *= 0.5e-3
    elif epoch > 40:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [79]:
def block_CNN(filters, ker, inpC): 
    """
    Returns CNN residual blocks
    """
    layer_1 = BatchNormalization()(inpC) 
    act_1 = Activation('relu')(layer_1) 

    conv_1 = Conv2D(filters, (ker-2, ker-2), padding = 'same')(act_1) 
    
    layer_2 = BatchNormalization()(conv_1) 
    act_2 = Activation('relu')(layer_2) 
  
    conv_2 = Conv2D(filters, (ker-2, ker-2), padding = 'same')(act_2) 
    return(conv_2)

In [80]:
def block_BiLSTM(inpR, filters, rnn_depth):
    """
    Returns LSTM residual blocks
    """
    x = inpR
    for i in range(rnn_depth):
        x_rnn = Bidirectional(LSTM(filters, return_sequences=True))(x)
        x_rnn = Dropout(0.7)(x_rnn)
        if i > 0 :
            x = add([x, x_rnn])
        else:
            x = x_rnn      
    return x

In [81]:
def model_cred(shape, filters):
    
    inp = Input(shape=shape, name='input')

    conv2D_2 = Conv2D(filters[0], (9,9), strides = (2,2), padding = 'same', activation = 'relu')(inp) 
    res_conv_2 = keras.layers.add([block_CNN(filters[0], 9, conv2D_2), conv2D_2]) 

    conv2D_3 = Conv2D(filters[1], (5,5), strides = (2,2), padding = 'same', activation = 'relu')(res_conv_2) 
    res_conv_3 = keras.layers.add([block_CNN(filters[1], 5, conv2D_3),conv2D_3]) 
    
    shape = K.int_shape(res_conv_3)   
    reshaped = Reshape((shape[1], shape[2]*shape[3]))(res_conv_3)
    
    res_BIlstm = block_BiLSTM(reshaped, filters = filters[3], rnn_depth = 2)
 
    UNIlstm = LSTM(filters[3], return_sequences=True)(res_BIlstm)
    UNIlstm = Dropout(0.8)(UNIlstm)  
    UNIlstm = BatchNormalization()(UNIlstm)
   
    dense_2 = TimeDistributed(Dense(filters[3], kernel_regularizer=l1(0.01), activation='relu'))(UNIlstm)
    dense_2 = BatchNormalization()(dense_2)
    dense_2 = Dropout(0.8)(dense_2)
    
    dense_3 = TimeDistributed(Dense(1, kernel_regularizer=l1(0.01), activation='sigmoid'))(dense_2)

    out_model = Model(inputs=inp, outputs=dense_3)
    return out_model

In [82]:
early_stopping_monitor = EarlyStopping(patience=5)
    
lr_scheduler = LearningRateScheduler(lr_schedule)
    
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5-2,
                                   min_lr=0.5e-7)

In [83]:
model = model_cred((248, 13, 3), filters = [8, 16, 32, 64, 128, 256])
model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(lr=lr_schedule(0)),
                  metrics=['binary_accuracy'])
model.summary()

Learning rate:  0.001
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 248, 13, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 124, 7, 8)    1952        ['input[0][0]']                  
                                                                                                  
 batch_normalization_36 (BatchN  (None, 124, 7, 8)   32          ['conv2d_36[0][0]']              
 ormalization)                                                                                    
                                                                      

In [84]:
seed_value = 15
tf.random.set_seed(seed_value)

In [85]:
checkpointer = ModelCheckpoint(filepath='model_best.hdf5',
                                       monitor='val_loss', verbose=0, mode='auto', save_best_only=True)
start_time = time.time()  
history = model.fit(
    X_1,
    y_1,
    epochs=200,
    batch_size=500,
    verbose=0,
    validation_data = (X1_val, y1_val),
    max_queue_size=5,
    callbacks = [checkpointer, lr_reducer, lr_scheduler, early_stopping_monitor] )
end_time = time.time()

Learning rate:  0.001


2022-03-06 16:40:56.367576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:40:59.714696: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:40:59.855412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:41:00.684557: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:41:00.766647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:41:02.567261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-06 16:41:03.922579: I tensorflow/core/grappler/optimizers/cust

Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  0.0001
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-05
Learning rate:  1e-06
Learning rate:  1e-06
Learning rate:  1e-06
Learning rate:  1e-06
Learning rate:  1e-06


In [86]:
history.history['val_binary_accuracy']

[0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913253784,
 0.9937084913

# Detection 

In [ ]:
def detection(st):
    '''
    ----------------------------------
    Input:
    st: the input should be a wave stream with the same scale of the training data, which is three chanel of the wave 
        , in which channel there are 2000 data points
    ----------------------------------
    Output:
    time: time detected time of p-wave arrival; and if there is no signal, print('No signal')
    '''
    # input process part
    st = slice_stream(st)
    st = st.filter("bandpass", freqmin=1, freqmax=45)
    st = st.resample(100)
    st = st.detrend("demean")
    st = st.normalize()
    
    # stft
    f, t, Zxx = signal.stft(st, fs = 100, nperseg=25)
    Zxx = np.abs(Zxx).T
    
    # input into model
    model.load_weights('model_best.hdf5')
    y = model.predict(Zxx)
    
    # time output
    if np.all(y == 0):
        print("No signal")
    else:
        t = np.argwhere(y==1)[0][0] ## get the index of the first '1' appear in the prediction
        lap = st[0].stats.endtime - st[0].stats.starttime
        arrival_time = st[0].stats.starttime + lap / 61 * (t-1)
        return arrival_time
        
        
        